In [1]:
import json, requests
import pandas as pd
from datetime import *
import time
from pyecharts.globals import ThemeType
# 只需要在顶部声明 CurrentConfig.ONLINE_HOST 即可
from pyecharts.globals import CurrentConfig,NotebookType
CurrentConfig.NOTEBOOK_TYPE= NotebookType.JUPYTER_NOTEBOOK


In [25]:
def get_base_info(oid):
    base_info_url = f'https://api.bilibili.com/x/web-interface/archive/stat?aid={oid}'
    base_info = requests.get(base_info_url, headers=dic_header).json()['data']
    # print(base_info) #可以输出转化为json形式的数据
    print('秒速五厘米：\n')
    print('播放数量：{}\n弹幕数量：{}\n收藏数量：{}\n硬币数量：{}\n分享数量：{}\n点赞数量：{}\n------\n评论数量：{}'.format(
        base_info['view'], base_info['danmaku'], base_info['favorite'],
        base_info['coin'], base_info['share'], base_info['like'], base_info['reply']
    ))

In [27]:
dic_header = {'User-Agent': 'Mozilla/5.0'}
oid = 135209
get_base_info(oid)

秒速五厘米：

播放数量：1407526
弹幕数量：73287
收藏数量：37942
硬币数量：4019
分享数量：2173
点赞数量：842
------
评论数量：2679


In [4]:
def get_comment_datas(oid):
    comment_url = 'https://api.bilibili.com/x/v2/reply'
    comment_page = 1
    comment_data_lst = []
    while True:
        try:
            param = {'callback': 'jQuery172002097397522090838_' + str(now_time),
                     'jsonp': 'jsonp',
                     'pn': comment_page,
                     'type': '1',
                     'oid': oid,
                     'sort': '2',
                     '_': now_time}

            html = requests.get(url=comment_url, headers=dic_header, params=param)
            start = html.text.index('{')
            end = html.text.index('})') + 1
            comment_data = json.loads(html.text[start:end])['data']['replies']
            # print(comment_data) #成功的转换为json数据
            print(f'当前正在爬取第{comment_page}页评论数据...')
            dic_coment = None
            for data in comment_data:
                dic_coment = {}
                dic_coment['member'] = data['member']['uname']
                dic_coment['like'] = data['like']
                dic_coment['comment'] = data['content']['message']
                dic_coment['time'] = datetime.fromtimestamp(data['ctime'])
                dic_coment['rpid'] = data['rpid_str']
                comment_data_lst.append(dic_coment)
                print('昵称: {}\n点赞数：{}\n'.format(dic_coment['member'],
                                                dic_coment['like']))
                # comment_data_lst.extend(get_reply_data(comment_page,dic_coment['rpid']))
                # 这个是下一步封装完爬取回复数据的函数后才添加的

            time.sleep(3)
            # 			if comment_page > 1:
            # 				break
            comment_page += 1

        except Exception as Comment_Page_Error:
            break

    return comment_data_lst

In [5]:
def get_reply_data(comment_page,rpid):
    reply_page = 1
    reply_data_lst =[]
    while True:
        print('正在爬取第{}页评论数据中的第{}页的回复数据......'.format(comment_page,reply_page))
        reply_url = 'https://api.bilibili.com/x/v2/reply/reply?&pn={}&type=1&oid=585951989&ps=10&root={}'.format(reply_page,rpid)
        html = requests.get(url=reply_url, headers=dic_header)
        reply_data = html.json()['data']['replies']
        try:
            for data in reply_data:
                dic_reply = {}
                dic_reply['comment'] = data['content']['message']
                dic_reply['member'] = data['member']['uname']
                dic_reply['like'] = data['like']
                dic_reply['time'] = datetime.fromtimestamp(data['ctime'])
                reply_data_lst.append(dic_reply)
                print('昵称: {}\n点赞数：{}\n'.format(dic_reply['member'],
                        dic_reply['like'] ))


# 			if reply_page > 1:
# 				break
            reply_page += 1
        except Exception as Reply_Page_Error:
            break

    return reply_data_lst

In [16]:
dic_header = {
    'User. Agent ': 'Mozilla/5.8',
    'Referer': 'https://www.bilibili.com/video/BV1Bz4y1z72s'
    }
# & type = 1 & oid = 585951989 & sort = 2 & _ = 1610032898130
oid = 135209  #秒速五厘米评论
now_time = int(time.time()*1000)
# get_ base. info(2616308358)
df = pd.DataFrame(get_comment_datas(oid))
df.to_csv('test1.csv', mode='a', index=False, sep=',', header=False, encoding="utf_8_sig")
print(df.head())
print(len(df))



当前正在爬取第1页评论数据...
昵称: 羽叶汐风
点赞数：1991

昵称: 小小彼此
点赞数：917

昵称: 筱原明里
点赞数：1077

昵称: 约谈人参
点赞数：132

昵称: 亚当斯密汤因比
点赞数：124

昵称: アタマワルイ
点赞数：95

昵称: 奶黄色窗帘
点赞数：80

昵称: 四只野猫
点赞数：67

昵称: 魔王saki
点赞数：66

昵称: 我爱发呆
点赞数：62

昵称: 奇妙子的脑洞
点赞数：48

昵称: Khabarovsk
点赞数：44

昵称: Lauvvvy
点赞数：34

昵称: 樱花缭乱A
点赞数：33

昵称: 左酱油油子
点赞数：31

昵称: smmog
点赞数：29

昵称: 干来哦
点赞数：29

昵称: 这个人现在有点迷
点赞数：28

昵称: 树裙子
点赞数：27

昵称: 二宮かずなり
点赞数：24

当前正在爬取第2页评论数据...
昵称: Kakascy
点赞数：23

昵称: 约谈人参
点赞数：23

昵称: 是阿乇啊
点赞数：21

昵称: Killersoap-
点赞数：20

昵称: 风雪桃妆
点赞数：20

昵称: 一般社员假面骑士
点赞数：19

昵称: hana_n_stanley
点赞数：18

昵称: 卤代烃不是烃
点赞数：17

昵称: 我所在的时光
点赞数：17

昵称: 不老の劍聖
点赞数：17

昵称: 阿木良良奈
点赞数：16

昵称: 皇家绅士凉水
点赞数：16

昵称: scereed
点赞数：15

昵称: 白山津
点赞数：15

昵称: triseasonal
点赞数：15

昵称: 白锦7
点赞数：14

昵称: 洛洛的小咲
点赞数：13

昵称: 只想一个人清静生活
点赞数：13

昵称: 仲月夜_
点赞数：13

昵称: 噗啊哈哈哈
点赞数：13

当前正在爬取第3页评论数据...
昵称: hikari_和光光
点赞数：13

昵称: bili_444607
点赞数：13

昵称: 情殇丿绕指柔
点赞数：13

昵称: 咖哩夜想曲
点赞数：13

昵称: Sorrow★锐
点赞数：12

昵称: 侑子小姐拜托l
点赞数：12

昵称: 北辰翠花
点赞数：12

昵称: 姐控眛控罗利控
点赞数：12

昵称: 落木_Durry
点赞数：12

昵称: 天然

当前正在爬取第23页评论数据...
昵称: 亚亚亚亚里沙
点赞数：2

昵称: Hayes00
点赞数：2

昵称: 打起精神不迷糊
点赞数：2

昵称: 芬达丶Fanta
点赞数：2

昵称: 李先生说营销
点赞数：2

昵称: 青菜萝卜番茄酱
点赞数：2

昵称: 奔向北河二
点赞数：2

昵称: 榴莲慕斯鱼子酱
点赞数：2

昵称: 故作镇定KCN
点赞数：2

昵称: 久坐不起m
点赞数：2

昵称: 萨克克
点赞数：2

昵称: lilithwitch
点赞数：2

昵称: 雪域圣灵
点赞数：2

昵称: 理事
点赞数：2

昵称: 拉面帅哥
点赞数：2

昵称: 超能发发
点赞数：2

昵称: EXsepe
点赞数：2

昵称: 钟楼小奶糕_
点赞数：2

昵称: 空想锁链
点赞数：2

昵称: 奥菲
点赞数：2

当前正在爬取第24页评论数据...
昵称: 邓恩辛
点赞数：2

昵称: 膽小鬼火箭彈
点赞数：2

昵称: _cardinal_
点赞数：2

昵称: 听雷雷
点赞数：2

昵称: 我脾气很暴的
点赞数：2

昵称: timothy2x
点赞数：2

昵称: AlexMercer先生
点赞数：2

昵称: Abyss_Gazer
点赞数：2

昵称: 整活の乔纳森
点赞数：2

昵称: 执灬念丶丶
点赞数：2

昵称: 黑泽优三
点赞数：2

昵称: 呆桃的小兔
点赞数：2

昵称: BillowSky
点赞数：2

昵称: 御风北斗
点赞数：2

昵称: 夏天夏天べ
点赞数：3

昵称: 贪睡的熊猫
点赞数：1

昵称: 千年风华
点赞数：1

昵称: 理外魔王拉提娜
点赞数：1

昵称: 晓美圆香
点赞数：1

昵称: 大才子小王
点赞数：1

当前正在爬取第25页评论数据...
昵称: TURMEN
点赞数：1

昵称: 玉老秃不秃
点赞数：1

昵称: 不不老爹
点赞数：1

昵称: 被安排的人
点赞数：1

昵称: 消停吧停停停
点赞数：1

昵称: bili_12358522
点赞数：1

昵称: 听说你很沙雕
点赞数：1

昵称: 訫疼
点赞数：1

昵称: 遗世而立的幻想乡
点赞数：1

昵称: 不见了秋
点赞数：1

昵称: 空城没有梦
点赞数：1

昵称: 4949449
点赞数：1

昵称: 萌动我世界
点赞数：1



当前正在爬取第45页评论数据...
昵称: 阿曦喜喜嘻嘻嘻
点赞数：0

昵称: 金剛丶
点赞数：0

昵称: 五月初夏Bilibili
点赞数：0

昵称: 丘陵零
点赞数：0

昵称: 奕修呀
点赞数：0

昵称: 西行妖の花
点赞数：0

昵称: 言叶不知桂
点赞数：0

昵称: Polevideo
点赞数：0

昵称: nellgod
点赞数：0

昵称: Egoits
点赞数：0

昵称: 女王女王蹦恰恰
点赞数：0

昵称: 帶帶小李
点赞数：0

昵称: 枇子菌
点赞数：0

昵称: 煎蛋的蛋
点赞数：0

昵称: 刈阿颃
点赞数：0

昵称: Di_Zang
点赞数：0

昵称: 最美不过是伪娘
点赞数：0

昵称: toyosang
点赞数：0

昵称: 伊万万万万万万万万
点赞数：0

昵称: 钢门大喷发
点赞数：0

当前正在爬取第46页评论数据...
昵称: 钢门大喷发
点赞数：0

昵称: 就是那灬么酷炫
点赞数：0

昵称: 桃气小咪
点赞数：0

昵称: 山猪娃
点赞数：0

昵称: 风格里哦
点赞数：0

昵称: 月兔和太阳丞
点赞数：0

昵称: 十六夜咲くや
点赞数：0

昵称: 浅羽衛
点赞数：0

昵称: 睿国无敌
点赞数：0

昵称: 虾壳侠
点赞数：0

昵称: 绝柒谙
点赞数：0

昵称: 暖暖↗
点赞数：0

昵称: 姐姐天生爱美丽
点赞数：0

昵称: 伊亚路
点赞数：0

昵称: 青蛙是大好人
点赞数：0

昵称: 呆萌の兔子酱
点赞数：0

昵称: 绅士的大魔王
点赞数：0

昵称: 茅屋为秋风所破歌
点赞数：0

昵称: XuTuT
点赞数：0

昵称: Blue丶悸动之心
点赞数：0

当前正在爬取第47页评论数据...
昵称: 樱花庄的喵污神
点赞数：0

昵称: 传说中的牛肉sama
点赞数：0

昵称: 云影cn
点赞数：0

昵称: 阿布拉夫斯基
点赞数：0

昵称: 微笑的魔理沙
点赞数：0

昵称: k-revolution
点赞数：0

昵称: Z050903100202
点赞数：0

昵称: 耳畔风痕
点赞数：0

昵称: 千冿澟
点赞数：0

昵称: 寒冰中的蛋蛋
点赞数：0

昵称: Y钊钊暮暮Y
点赞数：0

昵称: 寒冰中的蛋蛋
点赞数：0

昵称: 待君归来已陈年
点赞数：0

昵称

当前正在爬取第68页评论数据...
昵称: 熙夕言
点赞数：0

昵称: 驯兽带师金皮卡
点赞数：0

昵称: 吼姆啦荡漾
点赞数：0

昵称: 夏尔·凡多姆海恩_
点赞数：0

昵称: robbie1024
点赞数：0

昵称: 慈父刘俊
点赞数：0

昵称: 慈父刘俊
点赞数：0

昵称: 草莓圣代不加蛋黄酱
点赞数：0

昵称: ゚幽喵喵
点赞数：0

昵称: NiaNia丷
点赞数：0

昵称: 司马鸡
点赞数：0

昵称: 的我再546
点赞数：0

昵称: 选择成为偶像
点赞数：0

昵称: 发光的米饭
点赞数：0

昵称: 滓呀比947
点赞数：0

昵称: 煎茶coffee
点赞数：0

昵称: 垂我此194
点赞数：0

昵称: 长灯
点赞数：0

昵称: 私の猫を作ります
点赞数：0

昵称: 丛云鸢尾
点赞数：0

当前正在爬取第69页评论数据...
昵称: 雾雨谜拟咔丶
点赞数：0

昵称: 汐漌丶
点赞数：0

昵称: 金闪闪的光希
点赞数：0

昵称: Amadeus红筣栖
点赞数：0

昵称: 届不到的零度
点赞数：0

昵称: 饿了喝水渴了看书
点赞数：0

昵称: MK浩
点赞数：0

昵称: Niyoe
点赞数：0

昵称: 真·不知道
点赞数：0

昵称: 我爱宅妹
点赞数：0

昵称: 尐へ雨
点赞数：0

昵称: 莉莉佐目
点赞数：0

昵称: 木偶赏花
点赞数：0

昵称: Akari_w
点赞数：0

昵称: 心口难开「君莫泣」
点赞数：0

昵称: 桃酱neko
点赞数：0

昵称: Ayase-Chan
点赞数：0

昵称: 我爱CLANNAD
点赞数：0

昵称: kuduo
点赞数：0

昵称: 阳光下安静睡着的猫
点赞数：0

当前正在爬取第70页评论数据...
昵称: 阳光下安静睡着的猫
点赞数：0

昵称: 悦色az
点赞数：0

昵称: 南末次
点赞数：0

昵称: 櫻Trick
点赞数：0

昵称: 亅丿乛一乙
点赞数：0

昵称: 筷子为什么是直的
点赞数：0

昵称: 三枝花
点赞数：0

昵称: 僖窍扒592
点赞数：0

昵称: 映岳赶712
点赞数：0

昵称: 挪都剐029
点赞数：0

昵称: 炎zong
点赞数：0

昵称: 食豪沦622
点赞数：0

昵称: 罗L罗
点赞数：0

昵称

当前正在爬取第90页评论数据...
昵称: 达摩克利斯之剣
点赞数：0

昵称: 千纯
点赞数：0

昵称: 饭饭是粑粑
点赞数：0

昵称: Jilibili
点赞数：0

昵称: WDD少年@提不起劲
点赞数：0

昵称: WDD少年@提不起劲
点赞数：0

昵称: 北城尾巷茶微凉
点赞数：0

昵称: 大清月初六
点赞数：0

昵称: 给我秘密
点赞数：0

昵称: 一般通过天狗
点赞数：0

昵称: lAnguishl
点赞数：0

昵称: 月球上的人hdy
点赞数：0

昵称: -球球璇的肉肉日常-
点赞数：0

昵称: 麻糬使
点赞数：0

昵称: 学CS的猫
点赞数：0

昵称: 怪人之夜
点赞数：0

昵称: 酋崎Zzz
点赞数：0

昵称: 丶丶clcccccccccccc
点赞数：0

昵称: 玉老秃不秃
点赞数：0

昵称: 快樂三四
点赞数：0

当前正在爬取第91页评论数据...
昵称: 快樂三四
点赞数：0

昵称: 0818权先森
点赞数：0

昵称: 悲剧帝熊仔
点赞数：0

昵称: oktangrui
点赞数：0

昵称: 刹弑
点赞数：0

昵称: 迪酱
点赞数：0

昵称: 二宮米莉亚
点赞数：0

昵称: NiaNia丷
点赞数：0

昵称: 丶澜酱
点赞数：0

昵称: 晨安Chan
点赞数：0

昵称: 九毛鹿
点赞数：0

昵称: 咄夜未来
点赞数：0

昵称: LunaRowan
点赞数：0

昵称: 黑雪鸡
点赞数：0

昵称: 奥创
点赞数：0

昵称: 季秋十七
点赞数：0

昵称: Yomo酱
点赞数：0

昵称: 黑化の辉夜
点赞数：0

昵称: 老婆如伊芙
点赞数：0

昵称: 若灵似梦
点赞数：0

当前正在爬取第92页评论数据...
昵称: 糖醋猪骨头
点赞数：0

昵称: 南行going
点赞数：0

昵称: 南行going
点赞数：0

昵称: baijt123
点赞数：0

昵称: 墨染青丝
点赞数：0

昵称: 雾香·楼兰
点赞数：0

昵称: 柠檬皮
点赞数：0

昵称: 41MR_MAVE
点赞数：0

昵称: Bro丶J
点赞数：0

昵称: Bro丶J
点赞数：0

昵称: 抹茶的猫
点赞数：0

昵称: 不喜欢午睡
点赞数：0

昵称: Mobuu
点赞数：0

昵称: Vogue
点

In [2]:
import pandas as pd

data = pd.read_csv(r"test1.csv",sep=',',names=['nickname','likes','content','time','account'])

data.head()


,nickname,likes,content,time,account
0,羽叶汐风,1991,当初看这个是因为豆瓣上的几句话，现在再回味又是一番感觉了。\n每秒5厘米似乎这个速度并不是很...,2014-09-03 22:56:50,18301748
1,小小彼此,917,对秒五感触最深的人，也是对爱情最放不低的人，尤其是一段异地恋。\n 当...,2013-10-26 23:11:20,9414721
2,筱原明里,1077,顶着这个ID我觉得我得上来留个言什么的。,2012-07-28 23:52:55,3988460
3,约谈人参,132,普通：唉，晚了一步，终生的遗憾啊\n文艺：承诺输给了时间，心意败给了距离\n二逼：初恋被狗日...,2015-04-20 03:18:55,29383827
4,亚当斯密汤因比,124,我来泼冷水。从第二话开始男主对女主得就不再是爱了，之是一种执念而已，一种对于过去的执念和对自...,2015-06-06 04:41:41,32271853


In [3]:
data.describe()

,likes,account
count,2084.000000,2.084000e+03
mean,6.018714,1.142684e+08
std,75.962021,3.728182e+08
min,0.000000,1.862044e+06
25%,0.000000,1.700963e+07
50%,0.000000,2.931582e+07
75%,1.000000,7.304705e+07
max,1991.000000,3.911337e+09


In [4]:
data.dropna()

,nickname,likes,content,time,account
0,羽叶汐风,1991,当初看这个是因为豆瓣上的几句话，现在再回味又是一番感觉了。\n每秒5厘米似乎这个速度并不是很...,2014-09-03 22:56:50,18301748
1,小小彼此,917,对秒五感触最深的人，也是对爱情最放不低的人，尤其是一段异地恋。\n 当...,2013-10-26 23:11:20,9414721
2,筱原明里,1077,顶着这个ID我觉得我得上来留个言什么的。,2012-07-28 23:52:55,3988460
3,约谈人参,132,普通：唉，晚了一步，终生的遗憾啊\n文艺：承诺输给了时间，心意败给了距离\n二逼：初恋被狗日...,2015-04-20 03:18:55,29383827
4,亚当斯密汤因比,124,我来泼冷水。从第二话开始男主对女主得就不再是爱了，之是一种执念而已，一种对于过去的执念和对自...,2015-06-06 04:41:41,32271853
...,...,...,...,...,...
2079,阿木良良奈,16,一个没回头，一个没挽留,2016-01-27 23:46:48,81093318
2080,皇家绅士凉水,16,少年不识愁滋味，为赋新词强说愁。\n而今识尽愁滋味，却道天凉好个秋。,2016-09-21 06:59:29,145249685
2081,scereed,15,然而我并没有喜欢的人，亦没有人喜欢我，人生真是寂寞如雪啊,2015-05-07 17:41:27,30477162
2082,白山津,15,2017.5.28，日本时间凌晨2点52分。\n两年前的今天，看了秒速五厘米的第一遍，那个时...,2017-05-28 02:32:14,269245514


In [13]:
data1=data.drop_duplicates(ignore_index=True)

In [14]:
df1 = data1.sort_values(by="likes",ascending=False).head(5)
df1

,nickname,likes,content,time,account
0,羽叶汐风,1991,当初看这个是因为豆瓣上的几句话，现在再回味又是一番感觉了。\n每秒5厘米似乎这个速度并不是很...,2014-09-03 22:56:50,18301748
2,筱原明里,1077,顶着这个ID我觉得我得上来留个言什么的。,2012-07-28 23:52:55,3988460
1,小小彼此,917,对秒五感触最深的人，也是对爱情最放不低的人，尤其是一段异地恋。\n 当...,2013-10-26 23:11:20,9414721
3,约谈人参,132,普通：唉，晚了一步，终生的遗憾啊\n文艺：承诺输给了时间，心意败给了距离\n二逼：初恋被狗日...,2015-04-20 03:18:55,29383827
4,亚当斯密汤因比,124,我来泼冷水。从第二话开始男主对女主得就不再是爱了，之是一种执念而已，一种对于过去的执念和对自...,2015-06-06 04:41:41,32271853


In [15]:
from pyecharts import options as opts
from pyecharts.charts import Bar
from example.commons import Faker

In [16]:
c1 = (
    Bar()
    .add_xaxis(df1["content"].to_list())
    .add_yaxis("likes", df1["likes"].to_list(), color=Faker.rand_color())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="评论热度Top5"),
        datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
    )
    .render_notebook()
)
c1


In [20]:
# 评论里有键盘侠或者水军吗？

df_shuijun = data[['nickname','content']].groupby(by = 'nickname').count().sort_values(by = 'content',ascending = False)
df_shuijun

,content
nickname,
骑士姑娘I,10
NiaNia丷,8
风雪桃妆,7
勿莣初心,6
雾香·楼兰,6
...,...
冈碧洋子,1
兵封千里,1
关闭自爱症儿童,1


In [21]:
len(df_shuijun[df_shuijun['content'] >= 10])
#查看一下动态超过10条的数据量，也就是下面61的依据


1

In [23]:
shuijun_data_over_10 = df_shuijun.iloc[:61]
x = shuijun_data_over_10.index.tolist()
y = shuijun_data_over_10['content'].tolist()
#设置x，y数据
print(x,y)

['骑士姑娘I', 'NiaNia丷', '风雪桃妆', '勿莣初心', '雾香·楼兰', '没定拆哪儿', '冰冰のの', '樱学家与珂学家', 'Azarea_', '约谈人参', '黄瓜菌', '高远合欢', 'Mashiro.', 'Happy-iwnl-', '星沉雨', 'KuzSeKi', 'xrEkidas', 'Khabarovsk', '御风北斗', '远野貴樹', '新世纪基佬战士', '路.人', '喵之优雅', '风冈麻理桑', 'Silence鬼魅', '樱Hua树下', '美丽英俊', '我所在的时光', '秒速五厘米1988', '暴躁的西瓜01', '天空中的罗恩桑', '戏言|跟班', 'LY_ling', '一坨懒散的唐森', '二十三年禅丶', '110', '不正是我', '理外魔王拉提娜', '左酱油油子', '深の空', '玉老秃不秃', '余纪楠', 'Turimas', '被嫌弃的松子', '呀吼丶楠哥', '是阿乇啊', 'ミミツ', 'Tsuangri', '西煦', '是三水吉不是三水', '咪口桑', '不老の劍聖', '訫疼', '蜜汁甜不甜', '君心向山', 'WDD少年@提不起劲', 'WSの银', 'Synesthesia_', '暗竹Yam1Take', 'ZeroPassion', '最期之詩'] [10, 8, 7, 6, 6, 6, 6, 5, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [24]:
c2 = (
    Bar()
    .add_xaxis(x)
    .add_yaxis("content",y, color=Faker.rand_color())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="水军数据排行榜"),
        datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
    )
    .render_notebook()
)
c2